In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=CW>

In [3]:
from pyspark.sql import SparkSession
from preprocessing.Preprocessor import Preprocessor
import pyspark.sql.functions as F

In [4]:
pp = Preprocessor(spark)

In [5]:
train = pp.to_tfidf('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = False)
#train = train.sample(False,0.6)
test =  pp.to_tfidf('Cleaned Data/test.csv')
train.cache()
test.cache()

DataFrame[features: vector, label: int]

In [6]:
train.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1948,[0,1,2,3,4,...|    2|
|(1948,[0,1,6,7,16...|    0|
|(1948,[0,1,2,3,4,...|    0|
|(1948,[5,27,34,74...|    2|
|(1948,[0,1,2,4,5,...|    2|
+--------------------+-----+
only showing top 5 rows



In [10]:
train = train.toPandas()
test = test.toPandas()

In [17]:
import numpy as np
X_train = train['features'].apply(lambda x : np.array(x.toArray())).to_numpy().reshape(-1,1)

In [18]:
X_train = np.apply_along_axis(lambda x : x[0], 1, X_train)

In [21]:
Y_train = train['label']

In [22]:
from sklearn.ensemble import AdaBoostClassifier
AdaModel = AdaBoostClassifier(n_estimators=50, learning_rate = 1)
model = AdaModel.fit(X_train, Y_train)

In [23]:
X_test = test['features'].apply(lambda x : np.array(x.toArray())).to_numpy().reshape(-1,1)

In [24]:
X_test = np.apply_along_axis(lambda x : x[0], 1, X_test)

In [25]:
predictions = model.predict(X_test)

In [26]:
from sklearn.metrics import f1_score

In [27]:
Y_test = test['label']

In [30]:
f1_score(Y_test, predictions, average='micro')

0.718111389007503

In [32]:
"Refernces - http://blog.madhukaraphatak.com/spark-vector-to-numpy/"

'Refernces - http://blog.madhukaraphatak.com/spark-vector-to-numpy/'